In [27]:

# !pip install -q pymupdf easyocr pytesseract opencv-python pillow

# Windows에서 Tesseract가 설치돼 있는데 인식이 안 되면 아래 경로를 맞춰주세요.
# import pytesseract
# pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"
# !pip install pymupdf
# !pip install pymupdf easyocr pytesseract opencv-python

In [16]:
import re
import numpy as np
import cv2
from pathlib import Path
import pdfplumber
import easyocr
import pytesseract

## 1) PDF가 텍스트 PDF인지 먼저 판별 (OCR 스킵)

- 텍스트가 충분히 있으면: `read_text_pdf()`로 즉시 추출  
- 텍스트가 거의 없으면(스캔/이미지 PDF): OCR 파이프라인으로 진행


In [17]:
IMAGE_EXTS = {".jpg", ".jpeg", ".png", ".webp"}

def classify_for_ocr(path: str, min_chars: int = 30) -> dict:
    """
    Returns:
      - {"input_type": "image"}       : 이미지 파일 또는 이미지 PDF(스캔)
      - {"input_type": "text_pdf"}    : 텍스트가 충분한 PDF (OCR 불필요)
    """
    p = Path(path)
    ext = p.suffix.lower()

    if ext in IMAGE_EXTS:
        return {"input_type": "image"}

    if ext == ".pdf":
        doc = fitz.open(path)
        total = 0
        # 앞쪽 몇 페이지만 훑어도 충분한 경우가 많음
        for i in range(min(5, doc.page_count)):
            total += len((doc.load_page(i).get_text() or "").strip())
            if total >= min_chars:
                return {"input_type": "text_pdf"}
        return {"input_type": "image"}

    raise ValueError(f"지원하지 않는 파일 형식: {ext}")

def read_text_pdf(pdf_path: str) -> str:
    doc = fitz.open(pdf_path)
    chunks = []
    for i in range(doc.page_count):
        t = (doc.load_page(i).get_text() or "").strip()
        if t:
            chunks.append(t)
    return "\n\n".join(chunks)


## 2) EasyOCR Reader 캐시 (매번 초기화하면 매우 느림)

- `easyocr.Reader`는 생성 비용이 커서 **한 번만 만들고 재사용**합니다.


In [18]:
_EASYOCR_READER = None

def get_easyocr_reader(gpu: bool = False):
    global _EASYOCR_READER
    if _EASYOCR_READER is None:
        _EASYOCR_READER = easyocr.Reader(['ko', 'en'], gpu=gpu)
    return _EASYOCR_READER


## 3) OCR 품질 점수 (간단/빠르게)

- EasyOCR 결과가 애매할 때만 Tesseract를 fallback으로 돌리기 위해
  아주 간단한 heuristic score를 씁니다.


In [19]:
def normalize_spaces(s: str) -> str:
    s = s.replace("\u00a0", " ")
    s = re.sub(r"[ \t]+", " ", s)
    s = re.sub(r"\n{3,}", "\n\n", s)
    return s.strip()

def legal_cleanup_min(text: str) -> str:
    # 과한 후처리는 정보 손실 가능 → 최소만
    text = text.replace("’", "'").replace("“", '"').replace("”", '"')
    return text

def ocr_quality_score(text: str) -> float:
    """
    점수는 문서마다 다를 수 있어요. '상대 비교용'입니다.
    - 한글 비율, 법률 문서 키워드 히트, 줄 fragmentation 등을 반영
    """
    if not text:
        return 0.0
    n = len(text)
    kor = sum('가' <= c <= '힣' for c in text) / max(n, 1)
    legal_hits = len(re.findall(r"(제\s*\d+\s*조|시행령|시행규칙|대법원|임대인|임차인|보증금|계약)", text))
    # 줄이 너무 잘게 쪼개지면 감점
    frag = len([x for x in text.split("\n") if x.strip()]) / max(1, (n/80))
    score = 0.0
    score += kor * 2.0
    score += legal_hits * 0.4
    score += (1.0 - min(frag, 1.0)) * 1.0
    return float(score)


## 4) 최소 variant + 조건부 병용 OCR (이미지 파일용)

- variant는 **2개만**: `gray` / `otsu`
- EasyOCR로 먼저 평가 → best가 낮으면 그때만 Tesseract


In [20]:
def load_image_bgr(image_path: str) -> np.ndarray:
    # Windows 한글 경로를 위해 imdecode 사용
    img = cv2.imdecode(np.fromfile(image_path, dtype=np.uint8), cv2.IMREAD_COLOR)
    if img is None:
        img = cv2.imread(image_path)
    if img is None:
        raise FileNotFoundError(f"이미지를 읽을 수 없습니다: {image_path}")
    return img

def make_variants_fast(image_bgr: np.ndarray) -> dict:
    # 너무 큰 이미지는 OCR 시간 폭증 → 적당히 축소
    h, w = image_bgr.shape[:2]
    max_w = 1600
    if w > max_w:
        scale = max_w / w
        image_bgr = cv2.resize(image_bgr, (int(w*scale), int(h*scale)), interpolation=cv2.INTER_AREA)

    gray = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2GRAY)
    v1 = gray
    v2 = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]
    return {"v1_gray": v1, "v2_otsu": v2}

def run_easyocr_on_gray(gray: np.ndarray, gpu: bool = False) -> dict:
    reader = get_easyocr_reader(gpu=gpu)
    # detail=0이 더 빠름 (confidence 없음)
    txt = reader.readtext(gray, detail=0, paragraph=True)
    text = "\n".join(txt) if isinstance(txt, list) else str(txt)
    return {"text": text}

def run_tesseract_on_gray(gray: np.ndarray, psm: int = 6, oem: int = 3) -> dict:
    config = f"--oem {oem} --psm {psm}"
    text = pytesseract.image_to_string(gray, lang="kor+eng", config=config)
    return {"text": text}

def ocr_fast_image(image_path: str, gpu: bool = False, tesseract_fallback: bool = True, fallback_threshold: float = 1.0) -> dict:
    """
    1) 2개 variant로 EasyOCR 평가
    2) best_score < fallback_threshold 인 경우에만
       해당 best variant 1개만 Tesseract로 fallback 수행
    """
    bgr = load_image_bgr(image_path)
    variants = make_variants_fast(bgr)

    easy_candidates = []
    for vname, gray in variants.items():
        e = run_easyocr_on_gray(gray, gpu=gpu)
        e_text = legal_cleanup_min(normalize_spaces(e["text"]))
        e_score = ocr_quality_score(e_text)
        easy_candidates.append({"engine": "easyocr", "variant": vname, "text": e_text, "score": e_score})

    easy_best = max(easy_candidates, key=lambda x: x["score"]) if easy_candidates else None
    best = easy_best

    if tesseract_fallback and best is not None and best["score"] < fallback_threshold:
        gray = variants[best["variant"]]
        t = run_tesseract_on_gray(gray, psm=6, oem=3)
        t_text = legal_cleanup_min(normalize_spaces(t["text"]))
        t_score = ocr_quality_score(t_text)
        if t_score > best["score"]:
            best = {"engine": "tesseract", "variant": best["variant"], "text": t_text, "score": t_score}

    return {"best": best, "easy_candidates": sorted(easy_candidates, key=lambda x: x["score"], reverse=True)}


## 5) 이미지 PDF 빠르게 OCR: dpi=200 + 페이지 병렬 처리

- PDF를 페이지별로 렌더링해서 gray로 만들고
- 페이지별 OCR은 서로 독립이라 **ThreadPool**로 병렬 처리합니다.


In [21]:
from concurrent.futures import ThreadPoolExecutor

def pdf_to_grays(pdf_path: str, dpi: int = 200):
    doc = fitz.open(pdf_path)
    out = []
    for i in range(doc.page_count):
        page = doc.load_page(i)
        pix = page.get_pixmap(dpi=dpi)
        img = np.frombuffer(pix.samples, dtype=np.uint8).reshape(pix.height, pix.width, pix.n)
        if pix.n == 4:
            img = cv2.cvtColor(img, cv2.COLOR_BGRA2BGR)
        else:
            img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        out.append((i, gray))
    return out

def ocr_pdf_image_fast(pdf_path: str, gpu: bool = False, workers: int = 4, dpi: int = 200, fallback_threshold: float = 1.0) -> str:
    pages = pdf_to_grays(pdf_path, dpi=dpi)

    def ocr_one(item):
        i, gray = item
        # 2 variants
        v1 = gray
        v2 = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]
        variants = {"v1_gray": v1, "v2_otsu": v2}

        easy_candidates = []
        for vname, g in variants.items():
            e = run_easyocr_on_gray(g, gpu=gpu)
            e_text = legal_cleanup_min(normalize_spaces(e["text"]))
            e_score = ocr_quality_score(e_text)
            easy_candidates.append((e_score, e_text, vname))

        easy_candidates.sort(reverse=True, key=lambda x: x[0])
        best_score, best_text, best_v = easy_candidates[0] if easy_candidates else (0.0, "", "v1_gray")

        # conditional fallback
        if best_score < fallback_threshold:
            t = run_tesseract_on_gray(variants[best_v], psm=6, oem=3)
            t_text = legal_cleanup_min(normalize_spaces(t["text"]))
            t_score = ocr_quality_score(t_text)
            if t_score > best_score:
                best_text = t_text

        return i, best_text

    with ThreadPoolExecutor(max_workers=workers) as ex:
        results = list(ex.map(ocr_one, pages))

    results.sort(key=lambda x: x[0])
    return "\n\n".join([t for _, t in results if t.strip()])


## 6) 통합 함수: 파일 1개를 넣으면 알아서 처리

- 텍스트 PDF → 즉시 추출
- 이미지 PDF → OCR
- 이미지 파일 → OCR


In [22]:
def extract_text_fast(path: str, gpu: bool = False, workers: int = 4, dpi: int = 200, fallback_threshold: float = 1.0) -> dict:
    info = classify_for_ocr(path)
    if info["input_type"] == "text_pdf":
        text = read_text_pdf(path)
        return {"mode": "text_pdf", "text": text, "detail": info}

    # image input
    p = Path(path)
    if p.suffix.lower() == ".pdf":
        text = ocr_pdf_image_fast(path, gpu=gpu, workers=workers, dpi=dpi, fallback_threshold=fallback_threshold)
        return {"mode": "image_pdf_ocr", "text": text, "detail": info}
    else:
        ocr_res = ocr_fast_image(path, gpu=gpu, tesseract_fallback=True, fallback_threshold=fallback_threshold)
        text = ocr_res["best"]["text"] if ocr_res["best"] else ""
        return {"mode": "image_ocr", "text": text, "detail": ocr_res}


## 7) 실행 예시

In [26]:
%%time
# 파일 경로를 넣으세요
path = "data/210818_01 (1)_page-0002.jpg"

result = extract_text_fast(path, gpu=False, workers=4, dpi=200, fallback_threshold=1.0)
print("mode:", result["mode"])
print(result["text"][:1500])


mode: image_ocr
제6조(채무불이행과 손해배상) 당사자 일방이 채무릎 이행하지 아니하는 때에는 상대방은 상당한 기간을 정하여 그 이행울 최고하고 계약올 해제활 수 있으며 그로 인한 손해바상울 청구할 수 있다 . 다만 채무자가 미리 이행하지 아니활 의사틀 표시한 경우의 계약해 제는 최고클 요하지 아니한다 제7조(계약의 해지) @ 임차인은 본인의 과실 없이 임차주택의 일부가 멸실 기타 사유로 인하여 임대차의 목적대로 사용할 수 없는 경우어는 계약을 해지할 수 있다 임대인은 임차인이 2기1의 차임액에 달하도록 연체하거나 제4조 제시항올 위반한 경우 계약올 해지할 수 있다 제8조(경신요구와 거절) 0 임차인은 임대차기간이 끝나기 6개월 전부터 2개월 전까지의 기간에 계약강신올 요구할 수 있다 다만 임대인은 자신 또는 그 직계존속 직계비속의 실거주 등 주택임대 차보호범 제6조의3 제시항 각 호의 사유가 잎는 경우에 한하여 계약껑신의 요구릇 거절할 수 있다. 별지2 계약경신 거절봉지서 양식 사용 가능 임대인이 주택임대차보호법 제6조의3 제부항 제8호에 따른 실거주록 사유로 강신블 거절하없음에도 불구하고 경신 요구가 거절되지 아니하여더라면 강신되엇올 기간이 만료되기 전에 정당한 사유 없이 제3자에게 주택올 임대한 경우, 임대인은 경신거절로 인하여 임차인이 입은 손해틀 배상하여야 한다. 제2항에 따른 손해바상액은 주택임대 차보호법 제6조의3 제6항에 의한다. 제9조(계약의 종료) 임대차계약이 종료된 경우에 임차인은 임차주택올 원래의 상태로 복구하여 임대인에게 반환하고 이와 동시에 임대인은 보증금올 임차인에게 반환하여야 한다 . 다만 , 시설물의 노후화나 통상 생길 수 있는 파손 등은 임차인의 원상복구의무에 포함되지 아니하다_ 제 10조(비용의 정산) 0 임차인은 계약종료 시 공과금과 관리비틀 정산하여야 한다. 임차인은 이미 남부한 관리비 중 장기수선중당금을 임대인(소유자인 경우)에게 반환 청구활 수 있다. 다만 관리사무소 등 관리주체가 장기수선중당금을 정산하는 경